<a href="https://colab.research.google.com/github/kimdonggyu2008/deep_daiv_-/blob/main/hifigan_wgan_r1r2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install wavenet_vocoder
!pip install librosa==0.9.1
!pip install wandbㄹ
!pip install "pip<24.1"
!pip install git+https://github.com/One-sixth/fairseq.git

from google.colab import drive, output
drive.mount('/content/drive')
output.clear()

In [ ]:
%cd /content/drive/MyDrive/코딩공부/deep_daiv/PAPER_SUBMIT
# 본인 폴더 경로로 변경

/content/drive/MyDrive/코딩공부/deep_daiv/PAPER_SUBMIT


In [ ]:
!pip install --upgrade scoreq
!pip install torch-pesq
!pip install praat-parselmouth
output.clear()

In [ ]:
import os
import pickle
import numpy as np
import soundfile as sf
from scipy import signal
from scipy.signal import get_window
from librosa.filters import mel
from numpy.random import RandomState
import torch
import torch.nn as nn
from collections import OrderedDict
import torch.nn.functional as F
from torch.utils import data
from multiprocessing import Process, Manager
import easydict
from torch.backends import cudnn
import time
import datetime
from math import ceil
from tqdm import tqdm
from wavenet_vocoder import builder
from torch_pesq import PesqLoss
import scoreq

SCOREQ running on: cuda


/usr/local/lib/python3.11/dist-packages/fairseq/checkpoint_utils.py:322: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(f, map_location=torch.device("cpu")

In [ ]:
#utils
import glob
import os
import matplotlib
import torch
from torch.nn.utils import weight_norm
matplotlib.use("Agg")
import matplotlib.pylab as plt


def plot_spectrogram(spectrogram):
    fig, ax = plt.subplots(figsize=(10, 2))
    im = ax.imshow(spectrogram, aspect="auto", origin="lower",
                   interpolation='none')
    plt.colorbar(im, ax=ax)

    fig.canvas.draw()
    plt.close()

    return fig


def init_weights(m, mean=0.0, std=0.01):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        m.weight.data.normal_(mean, std)


def apply_weight_norm(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        weight_norm(m)


def get_padding(kernel_size, dilation=1):
    return int((kernel_size*dilation - dilation)/2)


def load_checkpoint(filepath, device):
    assert os.path.isfile(filepath)
    print("Loading '{}'".format(filepath))
    checkpoint_dict = torch.load(filepath, map_location=device)
    print("Complete.")
    return checkpoint_dict


def save_checkpoint(filepath, obj):
    print("Saving checkpoint to {}".format(filepath))
    torch.save(obj, filepath)
    print("Complete.")


def scan_checkpoint(cp_dir, prefix):
    pattern = os.path.join(cp_dir, prefix + '????????')
    cp_list = glob.glob(pattern)
    if len(cp_list) == 0:
        return None
    return sorted(cp_list)[-1]

In [ ]:
#env
import os
import shutil


class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self


def build_env(config, config_name, path): # env_path : cp_hifigan/config_v1.json에 저장
    t_path = os.path.join(path, config_name)
    if config != t_path:
        os.makedirs(path, exist_ok=True)
        shutil.copyfile(config, os.path.join(path, config_name))

In [ ]:
import math
import os
import random
import torch
import torch.utils.data
import numpy as np
from librosa.util import normalize
from scipy.io.wavfile import read
from librosa.filters import mel as librosa_mel_fn

MAX_WAV_VALUE = 32768.0

def load_wav(full_path):
    sampling_rate, data = read(full_path)
    return data, sampling_rate

def dynamic_range_compression(x, C=1, clip_val=1e-5):
    return np.log(np.clip(x, a_min=clip_val, a_max=None) * C)

def dynamic_range_decompression(x, C=1):
    return np.exp(x) / C

def dynamic_range_compression_torch(x, C=1, clip_val=1e-5):
    return torch.log(torch.clamp(x, min=clip_val) * C)

def dynamic_range_decompression_torch(x, C=1):
    return torch.exp(x) / C

def spectral_normalize_torch(magnitudes):
    output = dynamic_range_compression_torch(magnitudes)
    return output

def spectral_de_normalize_torch(magnitudes):
    output = dynamic_range_decompression_torch(magnitudes)
    return output

mel_basis = {}
hann_window = {}

def mel_spectrogram(y, n_fft, num_mels, sampling_rate, hop_size, win_size, fmin, fmax, center=False):
    if torch.min(y) < -1.:
        print('min value is ', torch.min(y))
    if torch.max(y) > 1.:
        print('max value is ', torch.max(y))

    global mel_basis, hann_window
    if fmax not in mel_basis:
        mel = librosa_mel_fn(sampling_rate, n_fft, num_mels, fmin, fmax)
        mel_basis[str(fmax)+'_'+str(y.device)] = torch.from_numpy(mel).float().to(y.device)
        hann_window[str(y.device)] = torch.hann_window(win_size).to(y.device)

    # Apply padding and calculate STFT
    y = torch.nn.functional.pad(y.unsqueeze(1), (int((n_fft-hop_size)/2), int((n_fft-hop_size)/2)), mode='reflect')
    y = y.squeeze(1)

    spec = torch.stft(y, n_fft, hop_length=hop_size, win_length=win_size, window=hann_window[str(y.device)],
                      center=center, pad_mode='reflect', normalized=False, onesided=True, return_complex=True)

    # STFT의 결과에서 절대값을 취해 크기만 남김
    spec = spec.abs()

    # spec의 차원이 (n_frames, frequency_bins) 형식으로 맞는지 확인
    if len(spec.shape) == 3:
        spec = spec.squeeze(1)

    # mel_basis와 spec의 차원이 맞는지 확인한 후 matmul 실행
    spec = torch.matmul(mel_basis[str(fmax)+'_'+str(y.device)], spec)
    spec = spectral_normalize_torch(spec)

    return spec

def get_dataset_filelist(a):
    with open(a.input_training_file, 'r', encoding='utf-8') as fi:
        training_files = [os.path.join(a.input_wavs_dir, x.split('|')[0] + '.wav')
                          for x in fi.read().split('\n') if len(x) > 0]

    with open(a.input_validation_file, 'r', encoding='utf-8') as fi:
        validation_files = [os.path.join(a.input_wavs_dir, x.split('|')[0] + '.wav')
                            for x in fi.read().split('\n') if len(x) > 0]
    return training_files, validation_files

class MelDataset(torch.utils.data.Dataset):
    def __init__(self, training_files, segment_size, n_fft, num_mels,
                 hop_size, win_size, sampling_rate,  fmin, fmax, split=True, shuffle=True, n_cache_reuse=1,
                 device=None, fmax_loss=None, fine_tuning=False, base_mels_path=None):
        self.audio_files = training_files
        random.seed(1234)
        if shuffle:
            random.shuffle(self.audio_files)
        self.segment_size = segment_size
        self.sampling_rate = sampling_rate
        self.split = split
        self.n_fft = n_fft
        self.num_mels = num_mels
        self.hop_size = hop_size
        self.win_size = win_size
        self.fmin = fmin
        self.fmax = fmax
        self.fmax_loss = fmax_loss
        self.cached_wav = None
        self.n_cache_reuse = n_cache_reuse
        self._cache_ref_count = 0
        self.device = device
        self.fine_tuning = fine_tuning
        self.base_mels_path = base_mels_path

    def __getitem__(self, index):
        filename = self.audio_files[index]
        if self._cache_ref_count == 0:
            audio, sampling_rate = load_wav(filename)
            audio = audio / MAX_WAV_VALUE
            if not self.fine_tuning:
                audio = normalize(audio) * 0.95
            self.cached_wav = audio
            if sampling_rate != self.sampling_rate:
                raise ValueError(f"{sampling_rate} SR doesn't match target {self.sampling_rate} SR")
            self._cache_ref_count = self.n_cache_reuse
        else:
            audio = self.cached_wav
            self._cache_ref_count -= 1

        audio = torch.FloatTensor(audio).unsqueeze(0)

        if not self.fine_tuning:  # self.fine_tuning : False
            if self.split:
                if audio.size(1) >= self.segment_size:
                    max_audio_start = audio.size(1) - self.segment_size
                    audio_start = random.randint(0, max_audio_start)
                    audio = audio[:, audio_start:audio_start+self.segment_size]
                else:
                    audio = torch.nn.functional.pad(audio, (0, self.segment_size - audio.size(1)), 'constant')

            mel = mel_spectrogram(audio, self.n_fft, self.num_mels,
                                  self.sampling_rate, self.hop_size, self.win_size, self.fmin, self.fmax,
                                  center=False)
        else:
            mel = np.load(
                os.path.join(self.base_mels_path, os.path.splitext(os.path.split(filename)[-1])[0] + '.npy'))
            mel = torch.from_numpy(mel)

            if len(mel.shape) < 3:
                mel = mel.unsqueeze(0)

            if self.split:
                frames_per_seg = math.ceil(self.segment_size / self.hop_size)

                if audio.size(1) >= self.segment_size:
                    mel_start = random.randint(0, mel.size(2) - frames_per_seg - 1)
                    mel = mel[:, :, mel_start:mel_start + frames_per_seg]
                    audio = audio[:, mel_start * self.hop_size:(mel_start + frames_per_seg) * self.hop_size]
                else:
                    mel = torch.nn.functional.pad(mel, (0, frames_per_seg - mel.size(2)), 'constant')
                    audio = torch.nn.functional.pad(audio, (0, self.segment_size - audio.size(1)), 'constant')

        mel_loss = mel_spectrogram(audio, self.n_fft, self.num_mels,
                                   self.sampling_rate, self.hop_size, self.win_size, self.fmin, self.fmax_loss,
                                   center=False)

        return (mel.squeeze(), audio.squeeze(0), filename, mel_loss.squeeze())

    def __len__(self):
        return len(self.audio_files)

## hifi-model

In [ ]:
#model
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.nn import Conv1d, ConvTranspose1d, AvgPool1d, Conv2d
from torch.nn.utils import weight_norm, remove_weight_norm, spectral_norm

LRELU_SLOPE = 0.1


class ResBlock1(torch.nn.Module):
    def __init__(self, h, channels, kernel_size=3, dilation=(1, 3, 5)):
        super(ResBlock1, self).__init__()
        self.h = h
        self.convs1 = nn.ModuleList([
            weight_norm(Conv1d(channels, channels, kernel_size, 1, dilation=dilation[0],
                               padding=get_padding(kernel_size, dilation[0]))),
            weight_norm(Conv1d(channels, channels, kernel_size, 1, dilation=dilation[1],
                               padding=get_padding(kernel_size, dilation[1]))),
            weight_norm(Conv1d(channels, channels, kernel_size, 1, dilation=dilation[2],
                               padding=get_padding(kernel_size, dilation[2])))
        ])
        self.convs1.apply(init_weights)

        self.convs2 = nn.ModuleList([
            weight_norm(Conv1d(channels, channels, kernel_size, 1, dilation=1,
                               padding=get_padding(kernel_size, 1))),
            weight_norm(Conv1d(channels, channels, kernel_size, 1, dilation=1,
                               padding=get_padding(kernel_size, 1))),
            weight_norm(Conv1d(channels, channels, kernel_size, 1, dilation=1,
                               padding=get_padding(kernel_size, 1)))
        ])
        self.convs2.apply(init_weights)

    def forward(self, x):
        for c1, c2 in zip(self.convs1, self.convs2):
            xt = F.leaky_relu(x, LRELU_SLOPE)
            xt = c1(xt)
            xt = F.leaky_relu(xt, LRELU_SLOPE)
            xt = c2(xt)
            x = xt + x
        return x

    def remove_weight_norm(self):
        for l in self.convs1:
            remove_weight_norm(l)
        for l in self.convs2:
            remove_weight_norm(l)


class ResBlock2(torch.nn.Module):
    def __init__(self, h, channels, kernel_size=3, dilation=(1, 3)):
        super(ResBlock2, self).__init__()
        self.h = h
        self.convs = nn.ModuleList([
            weight_norm(Conv1d(channels, channels, kernel_size, 1, dilation=dilation[0],
                               padding=get_padding(kernel_size, dilation[0]))),
            weight_norm(Conv1d(channels, channels, kernel_size, 1, dilation=dilation[1],
                               padding=get_padding(kernel_size, dilation[1])))
        ])
        self.convs.apply(init_weights)

    def forward(self, x):
        for c in self.convs:
            xt = F.leaky_relu(x, LRELU_SLOPE)
            xt = c(xt)
            x = xt + x
        return x

    def remove_weight_norm(self):
        for l in self.convs:
            remove_weight_norm(l)


class Generator(torch.nn.Module):
    def __init__(self, h):
        super(Generator, self).__init__()
        self.h = h
        self.num_kernels = len(h.resblock_kernel_sizes)
        self.num_upsamples = len(h.upsample_rates)
        self.conv_pre = weight_norm(Conv1d(80, h.upsample_initial_channel, 7, 1, padding=3))
        resblock = ResBlock1 if h.resblock == '1' else ResBlock2

        self.ups = nn.ModuleList()
        for i, (u, k) in enumerate(zip(h.upsample_rates, h.upsample_kernel_sizes)):
            self.ups.append(weight_norm(
                ConvTranspose1d(h.upsample_initial_channel//(2**i), h.upsample_initial_channel//(2**(i+1)),
                                k, u, padding=(k-u)//2)))

        self.resblocks = nn.ModuleList()
        for i in range(len(self.ups)):
            ch = h.upsample_initial_channel//(2**(i+1))
            for j, (k, d) in enumerate(zip(h.resblock_kernel_sizes, h.resblock_dilation_sizes)):
                self.resblocks.append(resblock(h, ch, k, d))

        self.conv_post = weight_norm(Conv1d(ch, 1, 7, 1, padding=3))
        self.ups.apply(init_weights)
        self.conv_post.apply(init_weights)

    def forward(self, x):
        x = self.conv_pre(x)
        for i in range(self.num_upsamples):
            x = F.leaky_relu(x, LRELU_SLOPE)
            x = self.ups[i](x)
            xs = None
            for j in range(self.num_kernels):
                if xs is None:
                    xs = self.resblocks[i*self.num_kernels+j](x)
                else:
                    xs += self.resblocks[i*self.num_kernels+j](x)
            x = xs / self.num_kernels
        x = F.leaky_relu(x)
        x = self.conv_post(x)
        x = torch.tanh(x)

        return x

    def remove_weight_norm(self):
        print('Removing weight norm...')
        for l in self.ups:
            remove_weight_norm(l)
        for l in self.resblocks:
            l.remove_weight_norm()
        remove_weight_norm(self.conv_pre)
        remove_weight_norm(self.conv_post)


class DiscriminatorP(torch.nn.Module):
    def __init__(self, period, kernel_size=5, stride=3, use_spectral_norm=False):
        super(DiscriminatorP, self).__init__()
        self.period = period
        norm_f = weight_norm if use_spectral_norm == False else spectral_norm
        self.convs = nn.ModuleList([
            norm_f(Conv2d(1, 32, (kernel_size, 1), (stride, 1), padding=(get_padding(5, 1), 0))),
            norm_f(Conv2d(32, 128, (kernel_size, 1), (stride, 1), padding=(get_padding(5, 1), 0))),
            norm_f(Conv2d(128, 512, (kernel_size, 1), (stride, 1), padding=(get_padding(5, 1), 0))),
            norm_f(Conv2d(512, 1024, (kernel_size, 1), (stride, 1), padding=(get_padding(5, 1), 0))),
            norm_f(Conv2d(1024, 1024, (kernel_size, 1), 1, padding=(2, 0))),
        ])
        self.conv_post = norm_f(Conv2d(1024, 1, (3, 1), 1, padding=(1, 0)))

    def forward(self, x):
        fmap = []

        # 1d to 2d
        b, c, t = x.shape
        if t % self.period != 0: # pad first
            n_pad = self.period - (t % self.period)
            x = F.pad(x, (0, n_pad), "reflect")
            t = t + n_pad
        x = x.view(b, c, t // self.period, self.period)

        for l in self.convs:
            x = l(x)
            x = F.leaky_relu(x, LRELU_SLOPE)
            fmap.append(x)
        x = self.conv_post(x)
        fmap.append(x)
        x = torch.flatten(x, 1, -1)

        return x, fmap


class MultiPeriodDiscriminator(torch.nn.Module):
    def __init__(self):
        super(MultiPeriodDiscriminator, self).__init__()
        self.discriminators = nn.ModuleList([
            DiscriminatorP(2),
            DiscriminatorP(3),
            DiscriminatorP(5),
            DiscriminatorP(7),
            DiscriminatorP(11),
        ])

    def forward(self, y, y_hat):
        y_d_rs = []
        y_d_gs = []
        fmap_rs = []
        fmap_gs = []
        for i, d in enumerate(self.discriminators):
            y_d_r, fmap_r = d(y)
            y_d_g, fmap_g = d(y_hat)
            y_d_rs.append(y_d_r)
            fmap_rs.append(fmap_r)
            y_d_gs.append(y_d_g)
            fmap_gs.append(fmap_g)

        return y_d_rs, y_d_gs, fmap_rs, fmap_gs


class DiscriminatorS(torch.nn.Module):
    def __init__(self, use_spectral_norm=False):
        super(DiscriminatorS, self).__init__()
        norm_f = weight_norm if use_spectral_norm == False else spectral_norm
        self.convs = nn.ModuleList([
            norm_f(Conv1d(1, 128, 15, 1, padding=7)),
            norm_f(Conv1d(128, 128, 41, 2, groups=4, padding=20)),
            norm_f(Conv1d(128, 256, 41, 2, groups=16, padding=20)),
            norm_f(Conv1d(256, 512, 41, 4, groups=16, padding=20)),
            norm_f(Conv1d(512, 1024, 41, 4, groups=16, padding=20)),
            norm_f(Conv1d(1024, 1024, 41, 1, groups=16, padding=20)),
            norm_f(Conv1d(1024, 1024, 5, 1, padding=2)),
        ])
        self.conv_post = norm_f(Conv1d(1024, 1, 3, 1, padding=1))

    def forward(self, x):
        fmap = []
        for l in self.convs:
            x = l(x)
            x = F.leaky_relu(x, LRELU_SLOPE)
            fmap.append(x)
        x = self.conv_post(x)
        fmap.append(x)
        x = torch.flatten(x, 1, -1)

        return x, fmap


class MultiScaleDiscriminator(torch.nn.Module):
    def __init__(self):
        super(MultiScaleDiscriminator, self).__init__()
        self.discriminators = nn.ModuleList([
            DiscriminatorS(use_spectral_norm=True),
            DiscriminatorS(),
            DiscriminatorS(),
        ])
        self.meanpools = nn.ModuleList([
            AvgPool1d(4, 2, padding=2),
            AvgPool1d(4, 2, padding=2)
        ])

    def forward(self, y, y_hat):
        y_d_rs = []
        y_d_gs = []
        fmap_rs = []
        fmap_gs = []
        for i, d in enumerate(self.discriminators):
            if i != 0:
                y = self.meanpools[i-1](y)
                y_hat = self.meanpools[i-1](y_hat)
            y_d_r, fmap_r = d(y)
            y_d_g, fmap_g = d(y_hat)
            y_d_rs.append(y_d_r)
            fmap_rs.append(fmap_r)
            y_d_gs.append(y_d_g)
            fmap_gs.append(fmap_g)

        return y_d_rs, y_d_gs, fmap_rs, fmap_gs


# hifi - train

In [ ]:
import wandb
os.environ["WANDB_API_KEY"] = "4c7d91ca2cd073dc0f1c148b6e4bacff713df5c6"
wandb.login()

wandb: Currently logged in as: jeo053472 (jeo053472-Chung-Ang University) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
# WandB 프로젝트 초기화
name = "WGAN_R1R2"   # ex) WGAN (노션 참고)
id = 'wgan_R1R2_1'     # 프로젝트마다 고유 id 부여 (실험마다 다르게 설정해야함, 만약 전 실험을 이어서 진행하고 싶다면 같은 id 기재)
wandb.init(project="WGAN_R1R2_1", name=name ,id = id, resume = 'allow')

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


## Loss

여기에서 Loss 방식 변경

In [ ]:
def feature_loss(fmap_r, fmap_g):
    loss = 0
    for dr, dg in zip(fmap_r, fmap_g):
        for rl, gl in zip(dr, dg):
            loss += torch.mean(torch.abs(rl - gl))

    return loss*2


# def discriminator_loss(disc_real_outputs, disc_generated_outputs): #원본
#     loss = 0
#     r_losses = []
#     g_losses = []
#     for dr, dg in zip(disc_real_outputs, disc_generated_outputs):
#         r_loss = torch.mean((1-dr)**2)
#         g_loss = torch.mean(dg**2)
#         loss += (r_loss + g_loss)
#         r_losses.append(r_loss.item())
#         g_losses.append(g_loss.item())

#     return loss, r_losses, g_losses

def discriminator_loss(disc_real_outputs, disc_generated_outputs): #wgan 버전
    loss = 0
    r_losses = []
    g_losses = []
    for dr, dg in zip(disc_real_outputs, disc_generated_outputs):
        r_loss = -torch.mean(dr)
        g_loss = torch.mean(dg)
        loss += (r_loss + g_loss)
        r_losses.append(r_loss.item())
        g_losses.append(g_loss.item())

    return loss, r_losses, g_losses


# def generator_loss(disc_outputs):#원본
#     loss = 0
#     gen_losses = []
#     for dg in disc_outputs:
#         l = torch.mean((1-dg)**2)
#         gen_losses.append(l)
#         loss += l

#     return loss, gen_losses

def generator_loss(disc_outputs): #wgan버전
    loss = 0
    gen_losses = []
    for dg in disc_outputs:
        l = -torch.mean(dg)
        gen_losses.append(l)
        loss += l

    return loss, gen_losses

def clip_weights(discriminator, clip_value):
    """
    Clips the weights of the discriminator to satisfy 1-Lipschitz constraint.
    """
    for p in discriminator.parameters():
        p.data.clamp_(-clip_value, clip_value)

In [ ]:
# #그래디언트 널티

# lambda_gp = 0.001

# def compute_gradient_penalty(discriminator, real_samples, fake_samples, device="cuda"):
#     """
#     Compute Gradient Penalty for WGAN-GP.
#     """
#     batch_size = real_samples.size(0)
#     epsilon = torch.rand(batch_size, 1, 1, 1, device=device)
#     interpolated = epsilon * real_samples + (1 - epsilon) * fake_samples
#     interpolated.requires_grad_(True)

#     interpolated_scores = discriminator(interpolated)
#     gradients = torch.autograd.grad(
#         outputs=interpolated_scores,
#         inputs=interpolated,
#         grad_outputs=torch.ones_like(interpolated_scores),
#         create_graph=True,
#         retain_graph=True
#     )[0]

#     gradients = gradients.view(batch_size, -1)
#     gradient_norm = gradients.norm(2, dim=1)
#     gradient_penalty = ((gradient_norm - 1) ** 2).mean()
#     return gradient_penalty


#penalty 계산 함수
def compute_r1_penalty(discriminators, real_samples, device, gamma=10.0):
    """
    Computes the R1 regularization penalty for each discriminator in the given discriminators list.
    Args:
        discriminators: A list of individual discriminators (DiscriminatorP or DiscriminatorS).
        real_samples: The real samples (y).
        device: The device (cuda or cpu).
        gamma: The regularization weight (default: 10.0).
    Returns:
        r1_penalty: The computed R1 penalty.
    """
    real_samples = real_samples.requires_grad_(True)  # Enable gradient tracking
    r1_penalties = []

    for d in discriminators:
        real_outputs, _ = d(real_samples)  # Get discriminator outputs for real samples
        grad_real = torch.autograd.grad(
            outputs=real_outputs.sum(),
            inputs=real_samples,
            create_graph=True, retain_graph=True, only_inputs=True
        )[0]  # Compute gradients

        r1_penalty = (grad_real.norm(2, dim=(1, 2)) ** 2).mean()  # Compute gradient norm squared and take mean
        r1_penalties.append(r1_penalty)

    return (gamma / 2) * sum(r1_penalties)  # Apply weighting factor

def compute_r2_penalty(discriminators, fake_samples, device, gamma=10.0):
    """
    Computes the R2 regularization penalty for each discriminator in the given discriminators list.
    Args:
        discriminators: A list of individual discriminators (DiscriminatorP or DiscriminatorS).
        fake_samples: The fake samples (generated by the generator).
        device: The device (cuda or cpu).
        gamma: The regularization weight (default: 10.0).
    Returns:
        r2_penalty: The computed R2 penalty.
    """
    fake_samples = fake_samples.requires_grad_(True)  # Enable gradient tracking
    r2_penalties = []

    for d in discriminators:
        fake_outputs, _ = d(fake_samples)  # Get discriminator outputs for fake samples
        grad_fake = torch.autograd.grad(
            outputs=fake_outputs.sum(),
            inputs=fake_samples,
            create_graph=True, retain_graph=True, only_inputs=True
        )[0]  # Compute gradients

        r2_penalty = (grad_fake.norm(2, dim=(1, 2)) ** 2).mean()  # Compute gradient norm squared and take mean
        r2_penalties.append(r2_penalty)

    return (gamma / 2) * sum(r2_penalties)  # Apply weighting factor


In [ ]:
from torchaudio.transforms import Resample
import tempfile
import librosa

def lsd(est ,target):
    assert est.shape == target.shape, "Spectrograms must have the same shape."
    est = est.squeeze(0).squeeze(0) ** 2
    target = target.squeeze(0).squeeze(0) ** 2
    # Compute the log of the magnitude spectrograms (adding a small epsilon to avoid log(0))
    epsilon = 1e-10
    log_spectrogram1 = torch.log10(target + epsilon)
    log_spectrogram2 = torch.log10(est + epsilon)
    squared_diff = (log_spectrogram1 - log_spectrogram2) ** 2
    squared_diff = torch.mean(squared_diff, dim = 1) ** 0.5
    lsd = torch.mean(squared_diff, dim = 0)
    return lsd

def lsd_hf(est, target, hf_ratio=0.25):
    assert est.shape == target.shape, "Spectrograms must have the same shape."
    est = est.squeeze(0).squeeze(0) ** 2
    target = target.squeeze(0).squeeze(0) ** 2

    # Define high-frequency range
    num_freq_bins = est.shape[0]
    hf_start = int(num_freq_bins * (1 - hf_ratio))  # Starting index for high frequencies

    # Focus on high-frequency bands
    est_hf = est[hf_start:, :]
    target_hf = target[hf_start:, :]

    # Compute the log of the magnitude spectrograms (adding a small epsilon to avoid log(0))
    epsilon = 1e-10
    log_spectrogram1 = torch.log10(target_hf + epsilon)
    log_spectrogram2 = torch.log10(est_hf + epsilon)
    squared_diff = (log_spectrogram1 - log_spectrogram2) ** 2
    squared_diff = torch.mean(squared_diff, dim=1) ** 0.5
    lsd_hf = torch.mean(squared_diff, dim=0)

    return lsd_hf

def extract_f0_from_audio(audio, sr, fmin=50, fmax=500):
    audio_np = audio.cpu().numpy()  # Convert to numpy for librosa
    f0, voiced_flag, _ = librosa.pyin(audio_np, fmin=fmin, fmax=fmax, sr=sr)
    f0 = torch.tensor(f0, dtype=torch.float32)  # Convert back to tensor
    f0[~torch.tensor(voiced_flag, dtype=torch.bool)] = 0  # Set unvoiced regions to 0
    return f0

def f0_rmse(f0_pred, f0_target):
    assert f0_pred.shape == f0_target.shape, "F0 shapes must match."
    squared_error = (f0_pred - f0_target) ** 2
    mse = torch.mean(squared_error)
    rmse = torch.sqrt(mse)
    return rmse

## train

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import itertools
import os
import time
import argparse
import json
import torch
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DistributedSampler, DataLoader
import torch.multiprocessing as mp
from torch.distributed import init_process_group
from torch.nn.parallel import DistributedDataParallel
import easydict
import torchaudio

torch.backends.cudnn.benchmark = True


def train(rank, a, h): # rank : 0
    # SCOREQ 초기화
    scoreq_nr = scoreq.Scoreq(data_domain='natural', mode='nr')  # No-reference

    # PESQ 초기화
    pesq = PesqLoss(0.5,h.sampling_rate).eval()
    for param in pesq.parameters():
      param.requires_grad = False

    # # lambda 값 설정
    # ex) loss_scale_formant = 0.001
    # ex) loss_scale_r1r2 = #

    if h.num_gpus > 1:
        init_process_group(backend=h.dist_config['dist_backend'], init_method=h.dist_config['dist_url'],
                           world_size=h.dist_config['world_size'] * h.num_gpus, rank=rank)

    torch.cuda.manual_seed(h.seed)
    device = torch.device('cuda:{:d}'.format(rank))

    generator = Generator(h).to(device)
    mpd = MultiPeriodDiscriminator().to(device)
    msd = MultiScaleDiscriminator().to(device)

    if rank == 0:
        os.makedirs(a.checkpoint_path, exist_ok=True)
        print("checkpoints directory : ", a.checkpoint_path)

    if os.path.isdir(a.checkpoint_path):                 # 만약 체크포인트 폴더에 g_ 또는 do_로 시작하는 파일이 있다면 체크포인트 로드됨.
        cp_g = scan_checkpoint(a.checkpoint_path, 'g_')
        cp_do = scan_checkpoint(a.checkpoint_path, 'do_')

    steps = 0
    if cp_g is None or cp_do is None:         # 처음부터 학습
        state_dict_do = None
        last_epoch = -1
    else:                                     # 체크포인트부터 학습
        state_dict_g = load_checkpoint(cp_g, device)
        state_dict_do = load_checkpoint(cp_do, device)
        generator.load_state_dict(state_dict_g['generator'])
        mpd.load_state_dict(state_dict_do['mpd'])
        msd.load_state_dict(state_dict_do['msd'])
        steps = state_dict_do['steps'] + 1
        last_epoch = state_dict_do['epoch']

    if h.num_gpus > 1:
        generator = DistributedDataParallel(generator, device_ids=[rank]).to(device)
        mpd = DistributedDataParallel(mpd, device_ids=[rank]).to(device)
        msd = DistributedDataParallel(msd, device_ids=[rank]).to(device)

    optim_g = torch.optim.AdamW(generator.parameters(), h.learning_rate, betas=[h.adam_b1, h.adam_b2])
    optim_d = torch.optim.AdamW(itertools.chain(msd.parameters(), mpd.parameters()),
                                h.learning_rate, betas=[h.adam_b1, h.adam_b2])

    if state_dict_do is not None:
        optim_g.load_state_dict(state_dict_do['optim_g'])
        optim_d.load_state_dict(state_dict_do['optim_d'])

    scheduler_g = torch.optim.lr_scheduler.ExponentialLR(optim_g, gamma=h.lr_decay, last_epoch=last_epoch)
    scheduler_d = torch.optim.lr_scheduler.ExponentialLR(optim_d, gamma=h.lr_decay, last_epoch=last_epoch)

    training_filelist, validation_filelist = get_dataset_filelist(a)

    trainset = MelDataset(training_filelist, h.segment_size, h.n_fft, h.num_mels,
                          h.hop_size, h.win_size, h.sampling_rate, h.fmin, h.fmax, n_cache_reuse=0,
                          shuffle=False if h.num_gpus > 1 else True, fmax_loss=h.fmax_for_loss, device=device,
                          fine_tuning=a.fine_tuning, base_mels_path=a.input_mels_dir)

    train_sampler = DistributedSampler(trainset) if h.num_gpus > 1 else None

    train_loader = DataLoader(trainset, num_workers=h.num_workers, shuffle=False,
                              sampler=train_sampler,
                              batch_size=h.batch_size,
                              pin_memory=True,
                              drop_last=True)

    if rank == 0:
        validset = MelDataset(validation_filelist, h.segment_size, h.n_fft, h.num_mels,
                              h.hop_size, h.win_size, h.sampling_rate, h.fmin, h.fmax, False, False, n_cache_reuse=0,
                              fmax_loss=h.fmax_for_loss, device=device, fine_tuning=a.fine_tuning,
                              base_mels_path=a.input_mels_dir)
        validation_loader = DataLoader(validset, num_workers=1, shuffle=False,
                                       sampler=None,
                                       batch_size=1,
                                       pin_memory=True,
                                       drop_last=True)

        sw = SummaryWriter(os.path.join(a.checkpoint_path, 'logs'))

    generator.train()
    mpd.train()
    msd.train()
    for epoch in range(max(0, last_epoch), a.training_epochs):  # 200 epoch
        if rank == 0:
            start = time.time()
            print("Epoch: {}".format(epoch+1))

        if h.num_gpus > 1:
            train_sampler.set_epoch(epoch)

        for i, batch in enumerate(train_loader):
            if rank == 0:
                start_b = time.time()
            x, y, _, y_mel = batch
            x = torch.autograd.Variable(x.to(device, non_blocking=True))
            y = torch.autograd.Variable(y.to(device, non_blocking=True))
            y_mel = torch.autograd.Variable(y_mel.to(device, non_blocking=True))
            y = y.unsqueeze(1)

            y_g_hat = generator(x)
            y_g_hat_mel = mel_spectrogram(y_g_hat.squeeze(1), h.n_fft, h.num_mels, h.sampling_rate, h.hop_size, h.win_size,
                                          h.fmin, h.fmax_for_loss)

            optim_d.zero_grad()

            # MPD
            y_df_hat_r, y_df_hat_g, _, _ = mpd(y, y_g_hat.detach())
            loss_disc_f, losses_disc_f_r, losses_disc_f_g = discriminator_loss(y_df_hat_r, y_df_hat_g)

            # MSD
            y_ds_hat_r, y_ds_hat_g, _, _ = msd(y, y_g_hat.detach())
            loss_disc_s, losses_disc_s_r, losses_disc_s_g = discriminator_loss(y_ds_hat_r, y_ds_hat_g)

            #R1 Penalty 추가
            # ✅ MPD & MSD 각각의 Discriminators 리스트를 전달
            r1_penalty_mpd = compute_r1_penalty(mpd.discriminators, y, device, gamma=1)
            r1_penalty_msd = compute_r1_penalty(msd.discriminators, y, device, gamma=1)

            r2_penalty_mpd = compute_r2_penalty(mpd.discriminators, y_g_hat.detach(), device, gamma=1)
            r2_penalty_msd = compute_r2_penalty(msd.discriminators, y_g_hat.detach(), device, gamma=1)

            # ✅ 최종 Discriminator Loss 계산 (R1 + R2 Penalty 포함)
            loss_disc_all = loss_disc_s + loss_disc_f + r1_penalty_mpd + r1_penalty_msd + r2_penalty_mpd + r2_penalty_msd
            wandb.log({"train/loss_disc_all" : loss_disc_all , "train/loss_disc_s" : loss_disc_s , "train/loss_disc_f" : loss_disc_f})

            loss_disc_all.backward()
            optim_d.step()

            # clip_weights(mpd, 0.01)
            # clip_weights(msd, 0.01)

            # Generator
            optim_g.zero_grad()

            # L1 Mel-Spectrogram Loss
            loss_mel = F.l1_loss(y_mel, y_g_hat_mel) * 45

            y_df_hat_r, y_df_hat_g, fmap_f_r, fmap_f_g = mpd(y, y_g_hat)
            y_ds_hat_r, y_ds_hat_g, fmap_s_r, fmap_s_g = msd(y, y_g_hat)
            loss_fm_f = feature_loss(fmap_f_r, fmap_f_g)
            loss_fm_s = feature_loss(fmap_s_r, fmap_s_g)
            loss_gen_f, losses_gen_f = generator_loss(y_df_hat_g)
            loss_gen_s, losses_gen_s = generator_loss(y_ds_hat_g)

            loss_gen_all = loss_gen_s + loss_gen_f + loss_fm_s + loss_fm_f + loss_mel   # generator loss 계산
            wandb.log({"train/loss_gen_all" : loss_gen_all , "train/loss_gen_s" : loss_gen_s , "train/loss_gen_f" : loss_gen_f,
                       "train/loss_fm_s" : loss_fm_s , "train/loss_fm_f" : loss_fm_f , "train/loss_mel" : loss_mel})

            loss_gen_all.backward()
            optim_g.step()

            if rank == 0:
                # STDOUT logging
                if steps % a.stdout_interval == 0:
                    with torch.no_grad():
                        mel_error = F.l1_loss(y_mel, y_g_hat_mel).item()

                    print('Steps : {:d}, Gen Loss Total : {:4.3f}, Mel-Spec. Error : {:4.3f}, s/b : {:4.3f}'.
                          format(steps, loss_gen_all, mel_error, time.time() - start_b))

                # checkpointing
                if steps % a.checkpoint_interval == 0 and steps != 0:
                    checkpoint_path = "{}/g_{:08d}".format(a.checkpoint_path, steps)
                    save_checkpoint(checkpoint_path,
                                    {'generator': (generator.module if h.num_gpus > 1 else generator).state_dict()})
                    checkpoint_path = "{}/do_{:08d}".format(a.checkpoint_path, steps)
                    save_checkpoint(checkpoint_path,
                                    {'mpd': (mpd.module if h.num_gpus > 1
                                                         else mpd).state_dict(),
                                     'msd': (msd.module if h.num_gpus > 1
                                                         else msd).state_dict(),
                                     'optim_g': optim_g.state_dict(), 'optim_d': optim_d.state_dict(), 'steps': steps,
                                     'epoch': epoch})

                # # Tensorboard summary logging
                # if steps % a.summary_interval == 0:
                #     sw.add_scalar("training/gen_loss_total", loss_gen_all, steps)
                #     sw.add_scalar("training/mel_spec_error", mel_error, steps)

                # Validation
                if steps % a.validation_interval == 0 and steps != 0:
                    generator.eval()
                    torch.cuda.empty_cache()

                    val_err_tot = 0
                    val_err_tot_rmse = 0
                    val_err_tot_f0rmse = 0
                    val_err_tot_lsd = 0
                    val_err_tot_lsd_hf = 0
                    val_err_tot_mos = 0

                    with torch.no_grad():
                        for j, batch in enumerate(validation_loader):
                            x, y, _, y_mel = batch
                            y_g_hat = generator(x.to(device))
                            y_mel = torch.autograd.Variable(y_mel.to(device, non_blocking=True))
                            y_g_hat_mel = mel_spectrogram(y_g_hat.squeeze(1), h.n_fft, h.num_mels, h.sampling_rate,
                                                          h.hop_size, h.win_size,
                                                          h.fmin, h.fmax_for_loss)
                            # L1 loss 계산
                            val_err_tot += F.l1_loss(y_mel, y_g_hat_mel).item()

                            # RMSE 계산
                            val_err_tot_rmse += torch.sqrt(F.mse_loss(y_g_hat_mel, y_mel, reduction='mean')).item()

                            # F0_RMSE 계산
                            f0_y = extract_f0_from_audio(y.squeeze(1), h.sampling_rate)
                            f0_y_g_hat = extract_f0_from_audio(y_g_hat.squeeze(1), h.sampling_rate)
                            val_err_tot_f0rmse += f0_rmse(f0_y_g_hat, f0_y).item()

                            # LSD
                            val_err_tot_lsd += lsd(y_g_hat_mel,  y_mel).item()

                            # LSD_HF
                            val_err_tot_lsd_hf += lsd_hf(y_g_hat_mel,  y_mel).item()

                            # SCOREQ 계산
                            # 참조 및 생성된 오디오를 파일로 저장 (SCOREQ는 파일 경로로 입력 받음)
                            test_audio_path = f"temp_test.wav"
                            torchaudio.save(test_audio_path, y_g_hat.squeeze(1).cpu(), h.sampling_rate)

                            # No-reference 모드에서 품질 평가
                            val_err_tot_mos += scoreq_nr.predict(test_path=test_audio_path).item()

                            if j <= 4:
                                if steps == 0:
                                    wandb.log({
                                        "Predicted Audio": wandb.Audio(
                                        y_g_hat.squeeze().cpu().numpy(),
                                        sample_rate=h.sampling_rate,
                                        caption="Predicted Audio"
                                    ),
                                    "Ground Truth Audio": wandb.Audio(
                                        y[0].squeeze().cpu().numpy(),
                                        sample_rate=h.sampling_rate,
                                        caption="Ground Truth Audio"
                                    )})

                                # sw.add_audio('generated/y_hat_{}'.format(j), y_g_hat[0], steps, h.sampling_rate)
                                y_hat_spec = mel_spectrogram(y_g_hat.squeeze(1), h.n_fft, h.num_mels,
                                                             h.sampling_rate, h.hop_size, h.win_size,
                                                             h.fmin, h.fmax)
                                wandb.log({'generated/y_hat_spec_{}'.format(j):
                                              plot_spectrogram(y_hat_spec.squeeze(0).cpu().numpy())})

                        val_err_l1 = val_err_tot / (j+1)
                        val_err_rmse = val_err_tot_rmse / (j+1)
                        val_err_f0rmse = val_err_tot_f0rmse / (j+1)
                        val_err_lsd = val_err_tot_lsd / (j+1)
                        val_err_lsd_hf = val_err_tot_lsd_hf / (j+1)
                        val_err_mos = val_err_tot_mos / (j+1)

                        wandb.log({"validation/val_err_l1": val_err_l1, "validation/val_err_rmse": val_err_rmse, "validation/val_err_f0rmse": val_err_f0rmse,
                                   "validation/val_err_lsd": val_err_lsd, "validation/val_err_lsd_hf": val_err_lsd_hf, "validation/val_err_mos": val_err_mos})

                    generator.train()

            steps += 1

        scheduler_g.step()
        scheduler_d.step()

        if rank == 0:
            print('Time taken for epoch {} is {} sec\n'.format(epoch + 1, int(time.time() - start)))


def main():
    print('Initializing Training Process..')

    a = easydict.EasyDict({
      "group_name" : None,
      "input_wavs_dir" : 'LJSpeech-1.1/wavs',
      "input_mels_dir" : None,                              # fine-tuning 안할거면 필요 없음
      "input_training_file" : 'LJSpeech-1.1/training.txt',
      "input_validation_file" : 'LJSpeech-1.1/validation.txt',
      "checkpoint_path" : 'cp_WGAN_R1R2_1',                     # 체크포인트 폴더명 지정 (cp_Wandb name이랑 똑같이 설정 -> ex) cp_WGAN )
      "config" : 'config_v1.json',
      "training_epochs" : 100,
      "stdout_interval" : 10,
      "checkpoint_interval" : 1000,
      "summary_interval" : 10,
      "validation_interval" : 1000,
      "fine_tuning" : False
      })


    with open(a.config) as f:
        data = f.read()

    json_config = json.loads(data)
    h = AttrDict(json_config)
    build_env(a.config, 'config.json', a.checkpoint_path)

    torch.manual_seed(h.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(h.seed)
        h.num_gpus = torch.cuda.device_count()
        h.batch_size = int(h.batch_size / h.num_gpus)
        print('Batch size per GPU :', h.batch_size)
    else:
        pass

    if h.num_gpus > 1:
        mp.spawn(train, nprocs=h.num_gpus, args=(a, h,))
    else:
        train(0, a, h)
        wandb.finish()

In [ ]:
import logging
logging.getLogger().setLevel(logging.ERROR)

In [ ]:
if __name__ == '__main__':
  # a = Args()
  main()

Initializing Training Process..
Batch size per GPU : 32
SCOREQ running on: cuda


/usr/local/lib/python3.11/dist-packages/hydra/experimental/initialize.py:43: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  deprecation_warning(message=message)
/usr/local/lib/python3.11/dist-packages/hydra/experimental/initialize.py:45: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  self.delegate = real_initialize(
/usr/local/lib/python3.11/dist-packages/hydra/experimental/compose.py:25: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  deprecation_warning(message=message)
/usr/local/lib/python3.11/dist-packages/hydra/core/default_element.py:124: UserWarning: In 'config': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/changes_to_package_header for more information
  deprecation_warning(
/usr/local/lib/python3.11/dist-pack

checkpoints directory :  cp_WGAN_R1R2_1
Epoch: 1
Steps : 0, Gen Loss Total : 88.566, Mel-Spec. Error : 1.957, s/b : 14.526
Steps : 10, Gen Loss Total : 64.227, Mel-Spec. Error : 1.419, s/b : 0.916
Steps : 20, Gen Loss Total : 51.304, Mel-Spec. Error : 1.128, s/b : 0.915
Steps : 30, Gen Loss Total : 47.509, Mel-Spec. Error : 1.043, s/b : 0.919
Steps : 40, Gen Loss Total : 47.356, Mel-Spec. Error : 1.036, s/b : 0.928
Steps : 50, Gen Loss Total : 44.342, Mel-Spec. Error : 0.966, s/b : 0.920
Steps : 60, Gen Loss Total : 46.255, Mel-Spec. Error : 1.011, s/b : 0.913
Steps : 70, Gen Loss Total : 38.763, Mel-Spec. Error : 0.843, s/b : 0.916
Steps : 80, Gen Loss Total : 40.091, Mel-Spec. Error : 0.869, s/b : 0.933
Steps : 90, Gen Loss Total : 40.147, Mel-Spec. Error : 0.868, s/b : 0.918
Steps : 100, Gen Loss Total : 38.383, Mel-Spec. Error : 0.832, s/b : 0.909
Steps : 110, Gen Loss Total : 37.898, Mel-Spec. Error : 0.822, s/b : 0.917
Steps : 120, Gen Loss Total : 37.985, Mel-Spec. Error : 0.824

In [ ]:
stop

# hifi-inference

In [ ]:
class Args:
        group_name = None
        input_wavs_dir = 'test_files'
        output_dir = 'generated_files'
        checkpoint_file = 'cp_in_hifigan/generator_v1.pth'



In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import glob
import os
import argparse
import json
import torch
from scipy.io.wavfile import write
# from env import AttrDict
# from meldataset import mel_spectrogram, MAX_WAV_VALUE, load_wav
# from models import Generator

h = None
device = None


def load_checkpoint(filepath, device):
    assert os.path.isfile(filepath)
    print("Loading '{}'".format(filepath))
    checkpoint_dict = torch.load(filepath, map_location=device)
    print("Complete.")
    return checkpoint_dict


def get_mel(x):
    return mel_spectrogram(x, h.n_fft, h.num_mels, h.sampling_rate, h.hop_size, h.win_size, h.fmin, h.fmax)


def scan_checkpoint(cp_dir, prefix):
    pattern = os.path.join(cp_dir, prefix + '*')
    cp_list = glob.glob(pattern)
    if len(cp_list) == 0:
        return ''
    return sorted(cp_list)[-1]


def inference(a):
    generator = Generator(h).to(device)

    state_dict_g = load_checkpoint(a.checkpoint_file, device)
    generator.load_state_dict(state_dict_g['generator'])

    filelist = os.listdir(a.input_wavs_dir)

    os.makedirs(a.output_dir, exist_ok=True)

    generator.eval()
    generator.remove_weight_norm()
    with torch.no_grad():
        for i, filname in enumerate(filelist):
            wav, sr = load_wav(os.path.join(a.input_wavs_dir, filname))
            wav = wav / MAX_WAV_VALUE
            wav = torch.FloatTensor(wav).to(device)
            x = get_mel(wav.unsqueeze(0))
            y_g_hat = generator(x)
            audio = y_g_hat.squeeze()
            audio = audio * MAX_WAV_VALUE
            audio = audio.cpu().numpy().astype('int16')

            output_file = os.path.join(a.output_dir, os.path.splitext(filname)[0] + '_generated.wav')
            write(output_file, h.sampling_rate, audio)
            print(output_file)


def main():
    print('Initializing Inference Process..')

    # parser = argparse.ArgumentParser()
    # parser.add_argument('--input_wavs_dir', default='test_files')
    # parser.add_argument('--output_dir', default='generated_files')
    # parser.add_argument('--checkpoint_file', required=True)
    # a = parser.parse_args()

    config_file = os.path.join(os.path.split(a.checkpoint_file)[0], 'config.json')
    with open(config_file) as f:
        data = f.read()

    global h
    json_config = json.loads(data)
    h = AttrDict(json_config)

    torch.manual_seed(h.seed)
    global device
    if torch.cuda.is_available():
        torch.cuda.manual_seed(h.seed)
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    inference(a)


if __name__ == '__main__':
    a = Args()
    main()


In [ ]:
#inference e2e
from __future__ import absolute_import, division, print_function, unicode_literals

import glob
import os
import numpy as np
import argparse
import json
import torch
from scipy.io.wavfile import write
# from env import AttrDict
# from meldataset import MAX_WAV_VALUE
# from models import Generator

h = None
device = None


def load_checkpoint(filepath, device):
    assert os.path.isfile(filepath)
    print("Loading '{}'".format(filepath))
    checkpoint_dict = torch.load(filepath, map_location=device)
    print("Complete.")
    return checkpoint_dict


def scan_checkpoint(cp_dir, prefix):
    pattern = os.path.join(cp_dir, prefix + '*')
    cp_list = glob.glob(pattern)
    if len(cp_list) == 0:
        return ''
    return sorted(cp_list)[-1]


def inference(a):
    generator = Generator(h).to(device)

    state_dict_g = load_checkpoint(a.checkpoint_file, device)
    generator.load_state_dict(state_dict_g['generator'])

    filelist = os.listdir(a.input_mels_dir)

    os.makedirs(a.output_dir, exist_ok=True)

    generator.eval()
    generator.remove_weight_norm()
    with torch.no_grad():
        for i, filname in enumerate(filelist):
            x = np.load(os.path.join(a.input_mels_dir, filname))
            x = torch.FloatTensor(x).to(device)
            y_g_hat = generator(x)
            audio = y_g_hat.squeeze()
            audio = audio * MAX_WAV_VALUE
            audio = audio.cpu().numpy().astype('int16')

            output_file = os.path.join(a.output_dir, os.path.splitext(filname)[0] + '_generated_e2e.wav')
            write(output_file, h.sampling_rate, audio)
            print(output_file)


def main():
    print('Initializing Inference Process..')

    parser = argparse.ArgumentParser()
    parser.add_argument('--input_mels_dir', default='test_mel_files')
    parser.add_argument('--output_dir', default='generated_files_from_mel')
    parser.add_argument('--checkpoint_file', required=True)
    a = parser.parse_args()

    config_file = os.path.join(os.path.split(a.checkpoint_file)[0], 'config.json')
    with open(config_file) as f:
        data = f.read()

    global h
    json_config = json.loads(data)
    h = AttrDict(json_config)

    torch.manual_seed(h.seed)
    global device
    if torch.cuda.is_available():
        torch.cuda.manual_seed(h.seed)
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    inference(a)


if __name__ == '__main__':
    main()
